In [1]:
# basics
import os
import sys
import time
import datetime   # 
import re

# load self-written func
sys.path.append("func/")      # add path 
import helper
import prompts

# load env variables
from dotenv import dotenv_values
ENV_VAR = dotenv_values("../env/.env")
Gemini_key = ENV_VAR['Gemini_key']
HF_key = ENV_VAR['HF_key']

# DS
import pandas as pd
import numpy as np

# DL
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM  # HF models
import google.generativeai as genai     # Gemini model
print(f"GPU detected: {torch.cuda.is_available()}")
print(f"GPU device count: {torch.cuda.device_count()}")

GPU detected: True
GPU device count: 1


In [5]:
# reload module if modified
from importlib import reload
reload(helper)

<module 'helper' from 'd:\\code\\quantization\\func\\helper.py'>

In [5]:
# get credentials
from huggingface_hub import login
login(token=HF_key)
genai.configure(api_key=Gemini_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\user\.cache\huggingface\token
Login successful


In [6]:
gemini = genai.GenerativeModel('gemini-pro')

## Get prompt arguments
* 30 types of conversation tone
* 50 types of restaurants

In [14]:
input_text = """
List 30 types of most common tone in conversation. Your response should have great variety and contain only the answer. Each tone should be separated by a new line
"""
generation_config = genai.GenerationConfig(temperature=1, top_k=32, top_p=0.3)
tones_str = gemini.generate_content(input_text, generation_config=generation_config)

In [51]:
tones = tones_str.text
tones = tones.replace('- ', '')
tones = tones.split('\n')
tones

['Assertive',
 'Aggressive',
 'Authoritative',
 'Calm',
 'Commanding',
 'Confident',
 'Conversational',
 'Critical',
 'Cynical',
 'Defensive',
 'Demeaning',
 'Dismissive',
 'Enthusiastic',
 'Excited',
 'Friendly',
 'Humorous',
 'Impersonal',
 'Indifferent',
 'Inquisitive',
 'Ironic',
 'Judgmental',
 'Neutral',
 'Polite',
 'Sarcastic',
 'Serious',
 'Submissive',
 'Sympathetic',
 'Understanding',
 'Urgent',
 'Poor English']

In [18]:
input_text = """
List 60 types of common restaurants. Your response should have great variety and contain only the answer. Each response should be separated by a new line
"""
food_str = gemini.generate_content(input_text)

In [1]:
food_types = food_str.text
food_types = re.sub('\d+\.\s', '', food_types)
food_types = food_types.split('\n')
food_types

In [7]:
input_text = prompts.restaurant_calib_data(tones[1], food_types[3:5])
print(input_text)



I'm working on building a chatbot for restaurant recommendations. Can you assist me in generating training data in the following format? 
Please create 20 conversations, each must contain user input and model outputs, and encapsulated within a Python dictionary. All conversations should be wrapped up into a Python list.

Ensure the conversations cover a variety of scenarios, preferences, and inquiries related to dining experiences, focusing on Brewery or Buffet restaurants.
The tone of user input should be Aggressive, and the sentence structure in each example should be different, 
reflecting different users and their unique styles of communication.

Remember to use double quotes ("") to enclose the text in the training data.

Example format:

[
  {
    "user": "Hi there! Wondering if you can help me find a spot for a cozy dinner tonight.",
    "AI assistant": "Of course! For a cozy dinner experience, I recommend [Restaurant Name]. They offer intimate ambiance and a menu filled with 

In [12]:
generation_config = genai.GenerationConfig(temperature=0.7, top_k=64)

calib_data_raw = {}   # buffer to save Gemini output:
i = 0
err = 0
start_time = time.time()
while len(calib_data_raw) < 30:
    tone = tones[i]
    food_type = food_types[i*2:2*(i+1)]
    try: 
        if tone not in calib_data_raw:
            print(f"{i}, Tone: {tone}, Food type: {str(food_type)}")
            input_text = prompts.restaurant_calib_data(tone=tone, food_type=food_type)
            output = gemini.generate_content(input_text, generation_config=generation_config)
            calib_data_raw[tone] = output.text   # save output
            i += 1
    except:   # may have internet error
        err += 1
        print(f"{err} Internet error at: {i}")
    print(f"Accumulated time spent: {(time.time() - start_time)/60:.4f} minutes")

0, Tone: Assertive, Food type: ['American', 'Barbecue']
Accumulated time spent: 0.3292 minutes
1, Tone: Aggressive, Food type: ['Breakfast', 'Brewery']
Accumulated time spent: 0.6882 minutes
2, Tone: Authoritative, Food type: ['Buffet', 'Burger joint']
Accumulated time spent: 1.0274 minutes
3, Tone: Calm, Food type: ['Cafe', 'Chinese']
Accumulated time spent: 1.3316 minutes
4, Tone: Commanding, Food type: ['Coffee shop', 'Creperie']
Accumulated time spent: 1.6456 minutes
5, Tone: Confident, Food type: ['Diner', 'Fast food']
Accumulated time spent: 1.9313 minutes
6, Tone: Conversational, Food type: ['Fine dining', 'Food truck']
Accumulated time spent: 2.2476 minutes
7, Tone: Critical, Food type: ['Gastropub', 'Greek']
Accumulated time spent: 2.5385 minutes
8, Tone: Cynical, Food type: ['Hamburger stand', 'Health food']
Accumulated time spent: 2.9127 minutes
9, Tone: Defensive, Food type: ['Hot dog stand', 'Indian']
Accumulated time spent: 3.2936 minutes
10, Tone: Demeaning, Food type: [

In [28]:
chat_lst = []
for key, val in calib_data_raw.items():
    val = val.replace("```", "")   # remove sep symbols
    val = val.replace("python", "")   # remove sep symbols
    chat_split = val.split(' = ')
    try:
        cur_lst = eval(chat_split[-1])
        cur_lst = [{"user_tone": key, **item} for item in cur_lst]
        chat_lst.extend(cur_lst)
    except:
        print(f"failed at key value: {key}")

In [49]:
calib_data = pd.DataFrame(chat_lst)
calib_data['user_input'] = np.where(calib_data['user'].isna(), calib_data['input'], calib_data['user'])
calib_data['model_output'] = np.where(calib_data['AI assistant'].isna(), calib_data['output'], calib_data['AI assistant'])
calib_data = calib_data[['user_tone', 'user_input', 'model_output']]
# calib_data.isna().mean()
now_timestamp = re.sub('(:|\s|\.)', '_', str(datetime.datetime.now()))
calib_data.to_csv(f"data/restaurant_chat_{now_timestamp}.csv", index=False, encoding="utf-8-sig")

In [64]:
pd.set_option('display.max_colwidth', None)
display(calib_data.sample(5))

,user_tone,user_input,model_output
455,Sarcastic,I'm looking for a place that's both kid-friendly and has a great selection of craft beers.,Kid-friendly and craft beers? [Restaurant Name] fits the bill. They have a dedicated kids' menu and an impressive tap list featuring local brews.
52,Authoritative,"Hi there, I'm in the mood for a buffet restaurant with a sushi bar.","Sushi lovers, unite! [Restaurant Name] combines a delectable buffet with a fully stocked sushi bar. Dive into a culinary adventure that will satisfy your cravings for both variety and freshness."
202,Demeaning,I'm in the mood for something healthy and light.,"For a healthy and refreshing Japanese meal, try [Restaurant Name]. They offer a variety of light dishes such as sashimi, salads, and steamed entrees."
460,Sarcastic,I'm looking for a place that serves up mouthwatering burgers and fries.,"Burgers and fries, you say? [Restaurant Name] is your burger heaven. Their patties are juicy, their fries are crispy, and their milkshakes are the perfect complement."
380,Ironic,I'm craving some Thai food that will make my taste buds dance.,Prepare for a culinary adventure at [Restaurant Name]. Their tantalizing dishes are a symphony of flavors that will awaken your senses and leave you craving more.


In [68]:
# push data onto huggingface:
from datasets import Dataset

finetuning_dataset = Dataset.from_pandas(calib_data)
finetuning_dataset.push_to_hub("tctsung/tctsung/chat_restaurant_recommendation")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/tctsung/gemini_restaurant_chat/commit/ad5409373efa4d37934ba38b67d2e2643ce60c59', commit_message='Upload dataset', commit_description='', oid='ad5409373efa4d37934ba38b67d2e2643ce60c59', pr_url=None, pr_revision=None, pr_num=None)

## Load an example model

In [69]:
# Open ELM model from Apple
model_id = "apple/OpenELM-270M-Instruct"
openelm_270m = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", force_download=True, trust_remote_code=True, padding_side="left")

c:\Users\user\anaconda3\envs\hf\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\user\anaconda3\envs\hf\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
save_folder = "../models/OpenELM-270M-Instruct"
openelm_270m.save_pretrained(save_folder)
tokenizer.save_pretrained(save_folder)

('../models/OpenELM-270M-Instruct\\tokenizer_config.json',
 '../models/OpenELM-270M-Instruct\\special_tokens_map.json',
 '../models/OpenELM-270M-Instruct\\tokenizer.json')

In [2]:
from transformers import GenerationConfig
generation_config = GenerationConfig(
	max_new_tokens=100, 
	do_sample=True,        # sampling or not, use greedy decoding if False
	temperature = 0.95
	# top_k = 30,          # default 50
	# top_p = 0.3          # default 1.0
)

In [14]:
# load saved models:
local_path = "../models/OpenELM-270M-Instruct"
openelm_270m = AutoModelForCausalLM.from_pretrained(local_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(local_path, trust_remote_code=True, padding='right')

c:\Users\user\anaconda3\envs\hf\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
# add padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

(2, 32000, 1)

In [11]:
input_text = "I'm craving some good ol' American comfort food. Any suggestions?"
device = torch.device('cuda')
output_text, generation_time = helper.generate(openelm_270m, tokenizer, input_text, generation_config, device=torch.device('cuda'))
print("---------------\n",output_text.split('<</SYS>>')[-1])

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


---------------
 

I'm craving some good ol' American comfort food. Any suggestions?[/INST]

## Favorite American recipes and dishes

 - Pulled Pork Sandwich (made by: TJ's): Roast beef, barbeque sauce, American mustard, onions, pickles, lettuce, and a bun. [/INST]

## Favourite American food culture moments

* [This family](https://youtu.be/9LKrRuTxbqY


In [70]:
# load llama3 model:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama3 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\hf\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [75]:
save_folder = "../models/Llama-3-8B-Instruct"
llama3.save_pretrained(save_folder, safe_serialization=False)
tokenizer.save_pretrained(save_folder)

('../models/Llama-3-8B-Instruct\\tokenizer_config.json',
 '../models/Llama-3-8B-Instruct\\special_tokens_map.json',
 '../models/Llama-3-8B-Instruct\\tokenizer.json')

In [87]:
input_text = "I'm craving some good ol' American comfort food. Any suggestions?"
device = torch.device('cuda')
output_text, generation_time = helper.generate(llama3, tokenizer, input_text, generation_config, device=torch.device('cuda'))
output_text

RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.